In [9]:
import pandas as pd
import psycopg2
import numpy as np
import re
from sqlalchemy import create_engine, text as sql_text
import datetime
from ydata_profiling import ProfileReport

# Projeto Analytics Engineering - Inside Airbnb

## PostgreSQL - Exportação dos dados "raw"

Conjunto de dados:  "Inside Airbnb"

Passo a Passo:
1. Vá no link https://insideairbnb.com/get-the-data
2. Procure por "Rio de Janeiro"
3. Baixe os 3 .csv: "Listing", "Reviews" e Calendar" 

0bs: Caso queira pular estes passos, basta utilizar os .csvs que já estão nessa pasta: https://drive.google.com/drive/folders/1pwCpVsMu5si7BX1lzWX3tipP3fAPuoX3?usp=drive_link

4. Crie uma pasta dentro desse repositorio como 'raw_data' e adicione os 3 .csv

### Conexão com PostgreSQL

In [47]:
# James:
# dbname='airbnb_project'
# user='postgres'
# host='localhost'
# password='admin'
# port='5432'

# Marcela: 
dbname='airbnb_project'
user='postgres'
host='localhost'
password='0125698741'
port='5433'

In [48]:
conn = psycopg2.connect(dbname=dbname, user=user, host=host, password=password, port=port)
db_params = {
    'dbname': dbname,
    'user': user,
    'host': host,
    'password': password,
    'port': port
}
db_url = f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}'
engine = create_engine(db_url)

### Criação das bases 'raw's
- Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") 

In [20]:
#path = 'dbt/analytics_eng_airbnb/raw_data/'
path = 'raw_data/'
df_listings_raw = pd.read_csv(path + "listings.csv")
df_reviews_raw = pd.read_csv(path + "reviews.csv")
df_calendar_raw = pd.read_csv(path + "calendar.csv")

In [21]:
df_listings_raw.to_sql('listings_raw', engine, if_exists='replace', index=False)
df_reviews_raw.to_sql('reviews_raw', engine, if_exists='replace', index=False)
df_calendar_raw.to_sql('calendar_raw', engine, if_exists='replace', index=False)

595

In [17]:
def load_data_from_table(table_name):
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql_query(query, conn)
    return df

In [7]:
profile = ProfileReport(df_listings_raw, title="Listings Report")
profile.to_file("resultados_listings.html")
profile = ProfileReport(df_reviews_raw, title="Reviews Report", minimal=True)
profile.to_file("resultados_reviews.html")
profile = ProfileReport(df_calendar_raw, title="Calendar Report")
profile.to_file("resultados_calendar.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

...

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## DBT - Inicialização

Passo a Passo:
1) Instale o DBT
- python -m pip install dbt-postgres
- pip install dbt


2) Vá até sua pasta "dbt" e rode o comando:
3) dbt init analytics_eng_airbnb 

OBS: você pode pular os passos 2 e 3 utilizando a pasta "dbt" dentro do arquivo do Github


Altere as credenciais em "profiles.yml"

# Camadas

## Bronze

Verifique os arquivos criados na pasta dbt/analytics_eng_airbnb/models/bronze
- source.yml com os sources de "listing_raw", "reviews_raw", "calendar_raw"

- models.yml com informações e verificações sobre as novas tabelas que serão criadas: "listing_bronze", "reviews_bronze", "calendar_bronze". Nesse arquivo, executamos:
    - Verificamos se ID em "listings" é unique e not null
    - Adicionamos referencias de "foreign key" para listings_id

- arquivos .sql para cada uma das 3 tabelas

Para executar as regras criadas e importar as tabelas _bronze para o seu PostgreSQL, entre na pasta dbt/analytics_eng_airbnb e rode o comando:

- dbt run --models bronze

## Silver

Verifique os arquivos criados na pasta dbt/analytics_eng_airbnb/models/silver
- source.yml com os sources de "listing_bronze", "reviews_bronze", "calendar_bronze"

- models.yml com informações e verificações sobre as novas tabelas que serão criadas: "listing_silver", "reviews_silver", "calendar_silver". Nesse arquivo, executamos:
    - X Y Z (Aqui, fale sobre qualidade de dados: integridade, precisão e consistencia para os dados da camada). Fale também quais métricas foram selecionadas para continuar em cada lista e o porque. E também sobre os testes feitos (garantindo que a camada atenda as regras de negocio e requisitos de qualidade). Utilize as colunas selecionadas por Andre, explique porque escolheram elas e adicione os testes ">0" (e outros que tem lá).

- arquivos .sql para cada uma das 3 tabelas. Nesses arquivos, executamos:
    - XYZ adicionar 

Para executar as regras criadas e importar as tabelas _silver para o seu PostgreSQL, entre na pasta dbt/analytics_eng_airbnb e rode o comando:

- dbt run --models silver

## Gold

Verifique os arquivos criados na pasta dbt/analytics_eng_airbnb/models/gold
- source.yml com os sources de "listing_silver", "reviews_silver", "calendar_silver"

- models.yml com informações e verificações sobre as novas tabelas que serão criadas:  "listing_gold", "reviews_gold", "calendar_gold". Nesse arquivo, executamos:
    - X Y Z (Aqui, fale sobre agregações aplicadas - quais fora e porquê. Nessa camada testes não precisam ser executados (acho)).

- arquivos .sql para cada uma das 3 tabelas. Nesses arquivos, executamos:
    - XYZ adicionar 

Para executar as regras criadas e importar as tabelas _gold para o seu PostgreSQL, entre na pasta dbt/analytics_eng_airbnb e rode o comando:

- dbt run --models gold